## Instructions {-}

1. This is the template for the code and final report on the Prediction Problem.

2. You may modify the template if you see fit, but it should have the information asked below.

In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.impute import KNNImputer
from sklearn.model_selection import RandomizedSearchCV

## 1) Data Preprocessing

Steps for cleaning and preparing the dataset

In [10]:
train_X = pd.read_csv("train_X.csv")
train_y = pd.read_csv("train_y.csv")
test = pd.read_csv('public_private_X.csv')
train = train_X.merge(train_y, on='ID')

In [11]:
relevant_predictors = train_X.columns.tolist()

In [12]:
values_to_remove = ['PRODUCT_MARKET', 'ID', 'PRODUCT_STATUS', 'PURCHASE_ORDER_DUE_DATE', 'ORDER_DATE', 'DIVISION_CODE', 'RESERVABLE_INDICATOR']
for value in values_to_remove:
    relevant_predictors.remove(value)

1. PRODUCT_MARKET: This was just the addition of PRODUCT_NUMBER and DIVISION_NUMBER so I thought it was repetitive.

2. ID: This is a unique value for each observation so this would not help the logistic regression.

3. PRODUCT_STATUS, RESERVABLE_INDICATOR: Both of them only have one unique value so they are not good predictors.

4. PURCHASE_ORDER_DUE_DATE, ORDER_DATE: There are other predictors that are related to dates that are more useful and numerical. In addition, the date range is very different between the train and test data so they won't be useful predictors.

5. DIVISION_CODE: This is the same variable as DIVISION_NUMBER so it is repetitive.

In [ ]:
# Used KNN imputation for imputing missing values
columns_to_impute = ['AVERAGE_DAILY_DEMAND_CASES', 'AVERAGE_VENDOR_ORDER_CYCLE_DAYS', 
                     'AVERAGE_ORDER_CYCLE_DAYS', 'AVERAGE_ORDER_CYCLE_CASES']

knn_imputer = KNNImputer(n_neighbors=5)

# Impute for train data
train_imputed = knn_imputer.fit_transform(train[columns_to_impute])
train[columns_to_impute] = pd.DataFrame(train_imputed, columns=columns_to_impute)

# Impute for test data
test_imputed = knn_imputer.transform(test[columns_to_impute])
test[columns_to_impute] = pd.DataFrame(test_imputed, columns=columns_to_impute)

## 2) Feature Engineering

Techniques used to create meaningful features

### 2-1. Log transformation

Applied log transformation to numerical predictors that were highly right skewed.

In [14]:
predictors = [
    'DISTANCE_IN_MILES', 
    'AVERAGE_PRODUCT_ORDER_QUANTITY_MARKET', 
    'ORDER_QUANTITY_DEVIATION', 
    'GIVEN_TIME_TO_LEAD_TIME_RATIO', 
    'AVERAGE_DAILY_DEMAND_CASES', 
    'AVERAGE_VENDOR_ORDER_CYCLE_DAYS', 
    'AVERAGE_ORDER_CYCLE_DAYS', 
    'AVERAGE_ORDER_CYCLE_CASES', 
    'LEAD_TIME_TO_DISTANCE_RATIO'
]

# Apply log1p transformation to all specified predictors
train[predictors] = np.log1p(train[predictors])
test[predictors] = np.log1p(test[predictors])

### 2-2. Binning

For numerical variables that had a cluster of extreme outliers, I binned them so they do not impact the regression.

In [15]:
bin_vars = [
    'TRANSIT_LEAD_TIME', 
    'PURCHASING_LEAD_TIME', 
    'DAYS_BETWEEN_ORDER_AND_DUE_DATE'
]

for var in bin_vars:
    train[f'{var}_BINNED'] = pd.qcut(train[var], q=8, labels=False, duplicates='drop')
    test[f'{var}_BINNED'] = pd.qcut(test[var], q=8, labels=False, duplicates='drop')

for var in bin_vars:
    relevant_predictors.remove(var)
    relevant_predictors.append(f'{var}_BINNED')

### 2-3. Categorical Variables

1. DIVISION_NUMBER

I've noticed that division number 102 and 100 hvae similar completion rates and assumed they behaved similarly. Therefore, I grouped them to reduce model complexity. The calculations are in the Interim Report.

In [16]:
train['DIVISION_NUMBER'] = train['DIVISION_NUMBER'].replace(102, 100)
test['DIVISION_NUMBER'] = test['DIVISION_NUMBER'].replace(102, 100)

2. ORDER_DAY_OF_WEEK 

I removed this variable since each day of the week had similar completion rates. I assumed it won't be helpful for the model.

In [17]:
relevant_predictors.remove("ORDER_DAY_OF_WEEK")

3. DUE_DATE_WEEKDAY

I've noticed that this variable had two main groups where one group had a high completion rate and the other with a lower completion rate. I divided them accordingly to reduce model complexity.

In [18]:
def categorize_day(day):
    if day in [1, 2, 4, 6]:
        return 0
    else:
        return 1

train['DUE_DATE_WEEKDAY_CATEGORY'] = train['DUE_DATE_WEEKDAY'].apply(categorize_day)
test['DUE_DATE_WEEKDAY_CATEGORY'] = test['DUE_DATE_WEEKDAY'].apply(categorize_day)

relevant_predictors.remove("DUE_DATE_WEEKDAY")
relevant_predictors.append('DUE_DATE_WEEKDAY_CATEGORY')

4. PURCHASE_ORDER_TYPE - Kept as it is

5. PRODUCT_CLASSIFICATION & PRODUCT_NUMBER

Both variables had high cardinality, but had varying completion rates for each group. However, due to the high number of unique values, I chose to do target encoding where each variable was replaced with their group's mean completion rate. I filtered out groups with less than 10 occurrences so they do not skew the distribution and imputed them with the global completion mean.

In [19]:
# Product Classification
product_counts = train['PRODUCT_CLASSIFICATION'].value_counts()

global_mean = train['ON_TIME_AND_COMPLETE'].mean()

product_group_means = train.groupby('PRODUCT_CLASSIFICATION')['ON_TIME_AND_COMPLETE'].mean()

train['PRODUCT_CLASSIFICATION_encoded'] = train['PRODUCT_CLASSIFICATION'].map(
    lambda x: product_group_means[x] if product_counts[x] >= 10 else global_mean
)

test['PRODUCT_CLASSIFICATION_encoded'] = test['PRODUCT_CLASSIFICATION'].map(
    lambda x: product_group_means.get(x, global_mean) if x in product_group_means else global_mean
)

# Product Number
product_counts = train['PRODUCT_NUMBER'].value_counts()

product_group_means = train.groupby('PRODUCT_NUMBER')['ON_TIME_AND_COMPLETE'].mean()

train['PRODUCT_NUMBER_encoded'] = train['PRODUCT_NUMBER'].map(
    lambda x: product_group_means[x] if product_counts[x] >= 10 else global_mean
)

test['PRODUCT_NUMBER_encoded'] = test['PRODUCT_NUMBER'].map(
    lambda x: product_group_means.get(x, global_mean) if x in product_group_means else global_mean
)

relevant_predictors.remove("PRODUCT_CLASSIFICATION")
relevant_predictors.append('PRODUCT_CLASSIFICATION_encoded')
relevant_predictors.remove("PRODUCT_NUMBER")
relevant_predictors.append('PRODUCT_NUMBER_encoded')

6. COMPANY_VENDOR_NUMBER & PURCHASE_FROM_VENDOR

Similarly, both variables had high cardinality, but varying completion rates, which is a good sign as a predictor. For similar reasons, I implemented target encoding, but in a slightly different manner. Since most of the groups had a reasonable amount of occurrences, I imputed with the average of the completion rate for each group and a smoothing factor so lower frequency groups do not skew the distribution.

In [20]:
m = 20

# Calculate category mean and global mean
category_mean = train.groupby('COMPANY_VENDOR_NUMBER')['ON_TIME_AND_COMPLETE'].mean()
global_mean = train['ON_TIME_AND_COMPLETE'].mean()
category_size = train['COMPANY_VENDOR_NUMBER'].value_counts()

train['COMPANY_VENDOR_NUMBER_ENCODED'] = train['COMPANY_VENDOR_NUMBER'].map(
    lambda x: ((category_mean.get(x, global_mean) * category_size.get(x, 0)) + (global_mean * m)) / 
              (category_size.get(x, 0) + m)
)

# Compute target encoding for test set and fill missing categories with global mean
test['COMPANY_VENDOR_NUMBER_ENCODED'] = test['COMPANY_VENDOR_NUMBER'].map(
    lambda x: ((category_mean.get(x, global_mean) * category_size.get(x, 0)) + (global_mean * m)) / 
              (category_size.get(x, 0) + m)
).fillna(global_mean)

# Calculate category mean and global mean
category_mean = train.groupby('PURCHASE_FROM_VENDOR')['ON_TIME_AND_COMPLETE'].mean()
global_mean = train['ON_TIME_AND_COMPLETE'].mean()
category_size = train['PURCHASE_FROM_VENDOR'].value_counts()

# Compute target encoding
train['PURCHASE_FROM_VENDOR_ENCODED'] = train['PURCHASE_FROM_VENDOR'].map(
    lambda x: ((category_mean.get(x, global_mean) * category_size.get(x, 0)) + (global_mean * m)) / 
              (category_size.get(x, 0) + m)
)

# Compute target encoding for test set and fill missing categories with global mean
test['PURCHASE_FROM_VENDOR_ENCODED'] = test['PURCHASE_FROM_VENDOR'].map(
    lambda x: ((category_mean.get(x, global_mean) * category_size.get(x, 0)) + (global_mean * m)) / 
              (category_size.get(x, 0) + m)
).fillna(global_mean)

relevant_predictors.remove("COMPANY_VENDOR_NUMBER")
relevant_predictors.append('COMPANY_VENDOR_NUMBER_ENCODED')
relevant_predictors.remove("PURCHASE_FROM_VENDOR")
relevant_predictors.append('PURCHASE_FROM_VENDOR_ENCODED')

7. SHIP_FROM_VENDOR

This was one of the most highly correlated categorical variables but target encoding would always remove this variable during lasso. Therefore, I binned into three groups according to the mean completion rates.

In [21]:
# I will divide the predictor into three groups using qcut
mean_div = train.groupby('SHIP_FROM_VENDOR', observed=False)['ON_TIME_AND_COMPLETE'].mean().reset_index()

mean_div['SHIP_FROM_VENDOR_GROUP'] = pd.qcut(mean_div['ON_TIME_AND_COMPLETE'], 
                                             q=[0, 0.3, 0.7, 1.0], 
                                             labels=['Low', 'Middle', 'High'], 
                                             duplicates='drop')

train = train.merge(mean_div[['SHIP_FROM_VENDOR', 'SHIP_FROM_VENDOR_GROUP']], on='SHIP_FROM_VENDOR', how='left')

test = test.merge(mean_div[['SHIP_FROM_VENDOR', 'SHIP_FROM_VENDOR_GROUP']], on='SHIP_FROM_VENDOR', how='left')

default_group = mean_div['SHIP_FROM_VENDOR_GROUP'].mode()[0]

# Assign the mode to NaN values in the test set
test['SHIP_FROM_VENDOR_GROUP'] = test['SHIP_FROM_VENDOR_GROUP'].fillna(default_group)

relevant_predictors.remove("SHIP_FROM_VENDOR")
relevant_predictors.append('SHIP_FROM_VENDOR_GROUP')

*All EDA and rationals for bin edges and grouping are in the Interim Report

## 3) Developing the Model

I created dummies for all categorical variables with the exception of SHIP_FROM_VENDOR_GROUP that had a strict ordinal order. Therefore, ordinal encoding was applied. For polynomial transformation, I assumed some numerical variables had a more complex relationship with the target, more than a linear relationship. Therefore, I chose degree=3 with the addition of interaction terms. I finally scaled everything at the end.

In [22]:
cat_var = train[relevant_predictors].select_dtypes(include='int').columns.tolist()
float_var = train[relevant_predictors].select_dtypes(include='float64').columns.tolist()

X_train = train[relevant_predictors].copy()
y_train = train_y['ON_TIME_AND_COMPLETE']
X_test = test[relevant_predictors].copy()

# 1. One-Hot Encode cat_var
X_train_onehot = pd.get_dummies(train[cat_var], columns=cat_var).astype(int)
X_test_onehot = pd.get_dummies(test[cat_var], columns=cat_var).astype(int)

# 2. Ordinal Encoding for SHIP_FROM_VENDOR_GROUP
encoder = OrdinalEncoder()

# Fit and transform training data
train['SHIP_FROM_VENDOR_GROUP'] = encoder.fit_transform(train[['SHIP_FROM_VENDOR_GROUP']])
train_encoded = pd.DataFrame(train['SHIP_FROM_VENDOR_GROUP'], columns=['SHIP_FROM_VENDOR_GROUP'])

# Transform test data
test['SHIP_FROM_VENDOR_GROUP'] = encoder.fit_transform(test[['SHIP_FROM_VENDOR_GROUP']])
test_encoded = pd.DataFrame(test['SHIP_FROM_VENDOR_GROUP'], columns=['SHIP_FROM_VENDOR_GROUP'])

# 3. Create interaction terms for float_var using PolynomialFeatures
poly = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False)
X_train_num = pd.DataFrame(poly.fit_transform(train[float_var]), columns=poly.get_feature_names_out(train[float_var].columns))
X_test_num = pd.DataFrame(poly.transform(test[float_var]), columns=poly.get_feature_names_out(train[float_var].columns))

# Combine both the one-hot encoded and polynomial features
X_train_combined = pd.concat([X_train_onehot, X_train_num, train_encoded], axis=1)
X_test_combined = pd.concat([X_test_onehot, X_test_num, test_encoded], axis=1)

# Apply StandardScaler
scaler = StandardScaler()
X_train_final = pd.DataFrame(scaler.fit_transform(X_train_combined), columns=X_train_combined.columns)
X_test_final = pd.DataFrame(scaler.transform(X_test_combined), columns=X_test_combined.columns)

## 4) Model Fine Tuning

Including cross-validation, regularization, and hyperparameter tuning, etc

I used Lasso to remove any irrelevant predictors and liblinear turned out to have a better overall accuracy score. The range of C values was chosen by guess and check where I learned that c values that were greater than 1 was too weak of a regularization (did not remove a lot of predictors). To reduce running time, I used RandomizedSearchCV.

In [23]:
# Define the Lasso model
model = LogisticRegression(penalty='l1', solver='liblinear', max_iter=3000, random_state=42)

# Define the parameter grid with a range of C values
param_dist = {'C': np.logspace(-3, 0, 20)}

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=10, 
    cv=5,     
    n_jobs=-1,  
    random_state=42,
    verbose=1
)

# Fit RandomizedSearchCV on the data
random_search.fit(X_train_final, y_train)

# Print the best hyperparameters and score
print(f"Best C value: {random_search.best_params_['C']}")
print(f"Best score: {random_search.best_score_}")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best C value: 0.6951927961775606
Best score: 0.7843530188087627


In [24]:
# Make Predictions
y_pred = random_search.predict(X_test_final)

# Save Predictions
test_ids = test['ID']
predictions_df = pd.DataFrame({'ID': test_ids, 'ON_TIME_AND_COMPLETE': y_pred})
predictions_df.to_csv('final_predictions.csv', index=False)

In [25]:
predictions_df['ON_TIME_AND_COMPLETE'].sum()

np.int64(5165)

Please note that your code should be reproducible, well-structured, and include clear comments for readability.
Your code should generate the csv file that you submitted.

This generated an accuracy of 76.4% with the public test data.

## 5) Key Takeaways (Short Reflection)

* Provide a brief summary of your key takeaways from this prediction problem.
* Reflect on challenges faced and lessons learned from your model building process.

The prediction problem was a very challenging yet fun project. I think the biggest takeaway I got was that data science and machine learning has no exact answers. I was surprised at how differently students approached this prediction problem and how many variations can happen within a single method. Unlike tests that have an exact answer, this problem only had a target threshold and we were allowed to use any methods or functions that we learned to achieve that in any way. This journey of finding my own unique path for the target threshold was difficult, but very rewarding. Another takeaway was that modeling is a very sensitive and keen process. Slight tweaks and a change in the tenth decimal place would fluctuate the accuracy scores a lot or an inclusion or exclusion of a variable signficantly affect the model performance.

This very reason made the model building process difficult for me. Since each model is so sensitive to the approach and even to a decimal point in a binning process, there was no clear answer or solution to make my model better. Although there are recommended steps to take, it turns out that some do not work for your model. This process was very frustrating since some proposed methods significantly made other students' models better, but not for mine. However, I learned that just keep trying new methods and new tweaks will eventually make the the model better. 